# Investigate

## Graph Investigattion

In [ ]:
import os
import onnx

model_name = "qwen25_vli_visual.cpu.float32.onnx-dynamo.onnx"
print(os.path.abspath(model_name))
model_onnx = onnx.load(model_name)

model_name_custom = "qwen25_vli_visual.cpu.float32.custom.onnx"
print(os.path.abspath(model_name_custom))
model_onnx_custom = onnx.load(model_name_custom)

In [ ]:
from onnx_diagnostic.helpers.onnx_helper import pretty_onnx

for node in model_onnx.graph.node:
    if node.name == "n16":
        print(pretty_onnx(node))
        g = node.attribute[0].g
        break

for node in model_onnx_custom.graph.node:
    if node.name == "n16":
        print(pretty_onnx(node))
        g_custom = node.attribute[0].g
        break

In [ ]:
g.input

In [ ]:
g.output

In [ ]:
g_custom.input

In [ ]:
g_custom.output

In [ ]:
def remove_inplace_body_last_input_output_type_for_loop(filename: str):
    model = onnx.load(filename, load_external_data=False)
    for node in model.graph.node:
        if node.op_type == "Loop":
            g = node.attribute[0].g
            g.input[-1].type.CopyFrom(onnx.TypeProto())
            g.output[-1].type.CopyFrom(onnx.TypeProto())
    onnx.save(model, filename, save_as_external_data=False)


remove_inplace_body_last_input_output_type_for_loop(model_name)

## Statistics

In [ ]:
import pandas

In [ ]:
index = [
    "model_id",
    "pretrained",
    "device",
    "dtype",
    "attention",
    "opset",
    "exporter",
]
values = [
    "abs",
    "%>0.1",
    "%>0.01",
    "export_duration",
    "speedup",
    "latency_torch",
    "latency_ort_n",
]
df = pandas.read_json("dump_models/collection_statistics.js", lines=True)
stat = df[[*index, *values]].groupby(index).max()
stat

In [ ]:
df = pandas.read_json("dump_models/collection_statistics.js", lines=True)
stat = df[df.exporter == "onnx-dynamo"][[*index, *values]].groupby(index).max()
stat